In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyabf
import os
import pandas as pd


In [2]:
file = r"C:\Users\dongq\OneDrive\mouse patch-seq\Mecp2 project\3 weeks\2023_05_05_0001.abf"

In [3]:
f = pyabf.ABF(file)


In [26]:
f.sweepPointCount

20000

In [37]:
for index in f.sweepList:
    f.setSweep(index)
    print(f.sweepEpochs.levels, f.sweepEpochs.types)

[0.0, 0.0, -100.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, -80.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, -60.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, -40.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, -20.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 0.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 20.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 40.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 60.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 80.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 100.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 120.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 140.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 160.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 180.0, 0.0] ['Step', 'Step', 'Step', 'Step']


In [38]:
f.sweepEpochs.p1s
f.sweepEpochs.p2s

[312, 1312, 11312, 20000]

In [39]:
f.setSweep(0)
first_epoc = f.sweepEpochs
f.setSweep(1)
second_epoc = f.sweepEpochs
dif = np.asarray(second_epoc.levels) - np.asarray(first_epoc.levels)



In [40]:
step = np.where(dif != 0)
start = np.asarray(first_epoc.p1s)[step][0]
end = np.asarray(first_epoc.p2s)[step][0]
(start, end)

(1312, 11312)

In [46]:
sampling_rate = f.sampleRate

In [47]:
from feature_extractor import SpikeFeatureExtractor, SpikeTrainFeatureExtractor
sfe = SpikeFeatureExtractor(filter=2)
spte = SpikeTrainFeatureExtractor(start=start/sampling_rate, end=end/sampling_rate)

In [41]:
import subthresh_features as sbth

In [50]:

rmp = []
tau = []
sag = []

t_set = []
i_set = []
v_set = []

temp_result_list = []
flag_spike = False
for index in f.sweepList:
    f.setSweep(index)
    t = f.sweepX
    v = f.sweepY
    i = f.sweepC

    if f.sweepUnitsY == f.sweepUnitsC:
        v = v/20
    current = np.asarray(f.sweepEpochs.levels)[step][0]

    rmp.append(np.median(v[np.where(i == 0)]))
    if current < 0:
        tau.append(sbth.time_constant(t=t, v=v, i=i, start=start/sampling_rate, end=(end-2)/sampling_rate))
        sag.append(sbth.sag(t=t, v=v, i=i, start=start/sampling_rate, end=end/sampling_rate))
        t_set.append(t)
        i_set.append(i)
        v_set.append(v)
    
    if current == 0:
        t_set.append(t)
        i_set.append(i)
        v_set.append(v)

    ft = sfe.process(t, v, i)
    # ft.to_csv(f'{index}.csv', index=False)
    sptft= spte.process(t=t, v=v, i=i, spikes_df=ft)
    
    sptft['current'] = current
    temp_result_list.append((ft, sptft))

rin = sbth.input_resistance(t_set=t_set, i_set=i_set, v_set=v_set, start=start/sampling_rate, end=end/sampling_rate)

  


In [43]:
1508+1311


2819

## Todo:
1. start, end time
2. sag ratio
3. volatge clamp mode